In [1]:
import torch
import torch_geometric as tg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle as pkl
import scipy

device = torch.device("cpu")

/opt/homebrew/Caskroom/miniforge/base/envs/me555/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "./Data/output/"
n_sample = 30

input_name = "input_dict_"
solution_name = "solution_dict_"

input_dict_list = []
solution_dict_list = []

# read input, solution, and indices dict pkl file from 1 to n_sample
for i in range(1, n_sample + 1):
    with open(data_path + input_name + str(i) + ".pkl", "rb") as f:
        input_dict_list.append(pkl.load(f))

    with open(data_path + solution_name + str(i) + ".pkl", "rb") as f:
        solution_dict_list.append(pkl.load(f))

with open(data_path + "indices_dict.pkl", "rb") as f:
    indices_dict = pkl.load(f)

In [3]:
input_dict_list[0]["A"]

<1119720x577440 sparse matrix of type '<class 'numpy.float64'>'
	with 2392970 stored elements in Compressed Sparse Row format>

In [4]:
n_vars = input_dict_list[0]["A"].shape[1]
n_cons = input_dict_list[0]["A"].shape[0]

N = n_vars + n_cons

print(N)

1697160


In [5]:
# convert A matrix of bipartite data to COO format

input_data_dict = []

for i in range(n_sample):

    tmp_dict = {}

    # for row in range(n_vars):
    #     for col in range(n_cons):
    #         if input_dict_list[i]["A"][row, col] != 0:
    #             adj_matrix[row, n_vars + col] = input_dict_list[i]["A"][row, col]
    #             adj_matrix[n_vars + col, row] = input_dict_list[i]["A"][row, col]

    I, J, V = scipy.sparse.find(input_dict_list[i]["A"])
    # adj_matrix[I, n_vars + J] = V
    # adj_matrix[n_vars + J, I] = V

    # # convert to COO format
    edge_index = torch.stack([torch.tensor(I), torch.tensor(n_cons + J)], dim=0)

    # expand V to 2D
    edge_attr = torch.tensor(V).unsqueeze(1)

    tmp_dict["edge_index"] = edge_index
    tmp_dict["edge_attr"] = edge_attr

    input_data_dict.append(tmp_dict)


In [6]:
# print shape of edge_index and edge_attr
print("shape of edge_index: ", input_data_dict[0]["edge_index"].shape)
print("shape of edge_attr: ", input_data_dict[0]["edge_attr"].shape)

shape of edge_index:  torch.Size([2, 2392970])
shape of edge_attr:  torch.Size([2392970, 1])


In [7]:
print(input_dict_list[0]["b"].shape)
print(input_dict_list[0]["c"].shape)

(1119720,)
(577440,)


In [8]:
# node features
for i in range(n_sample):
    input_data_dict[i]["x"] = torch.cat(
        [torch.tensor(input_dict_list[i]["b"]), torch.tensor(input_dict_list[i]["c"])]
    )

    # expand dimension
    input_data_dict[i]["x"] = input_data_dict[i]["x"].unsqueeze(1)    

In [9]:
# solution dict keys
print(solution_dict_list[0].keys())

dict_keys(['DayAheadBuySellStatus', 'DayAheadOnOffChargingStatus', 'DayAheadChargingPower', 'DayAheadUtilityPowerOutput', 'SOC', 'output'])


In [10]:
input_dict_list[0]["A"].shape

(1119720, 577440)

In [11]:
print(indices_dict.keys())
DayAheadBuySellIndices = indices_dict["DayAheadBuySellStatus"]
DayAheadOnOffChargingStatusIndices = indices_dict["DayAheadOnOffChargingStatus"]

dict_keys(['DayAheadBuySellStatus', 'DayAheadOnOffChargingStatus', 'DayAheadChargingPower', 'DayAheadUtilityPowerOutput', 'SOC'])


In [26]:
# variable nodes are the last n_vars nodes
# one batch has N nodes, if batch size is 1, then batch of x = [N]
# if batch size is 2, then batch of x has shape [N + N]
# therefore for batch size of 2, the variable nodes are located in range(n_cons + N, 2 * N)
# for batch size of 3, the variable nodes are located in range(n_cons + 2 * N, 3 * N)
# and so on
variable_nodes = []
BATCH_SIZE = 2
variable_nodes = [
    range(n_cons + i * N, (i + 1) * N) for i in range(BATCH_SIZE)]

In [49]:
test = [0, 1, 2, 3]
test[-1] - test[0]

3

In [75]:
n_cons_test = 1
n_vars_test = 3

cons_test = [0, 1, 2]

start_index = n_cons_test

np.array(range(start_index + cons_test[0], n_cons_test + cons_test[0] + (cons_test[-1] - cons_test[0]) + 1))

array([1, 2, 3])

In [81]:
# variable nodes are located at range(n_cons + i * N, (i + 1) * N) for i in range(BATCH_SIZE)
# DayAheadBuySellIndices are located at range(n_cons + i * N + DayAheadBuySellIndices[0], n_cons + i * N + DayAheadBuySellIndices[1]) for i in range(BATCH_SIZE)

# test if variable nodes are located at the correct position
BATCH_SIZE = 2
DayAheadBuySellNodes = [
    range(n_cons + i * N + DayAheadBuySellIndices[0], n_cons + i * N + DayAheadBuySellIndices[0] + (DayAheadBuySellIndices[-1] - DayAheadBuySellIndices[0]) + 1) for i in range(BATCH_SIZE)]

assert np.array_equal(n_cons + np.array(DayAheadBuySellIndices), np.array(DayAheadBuySellNodes[0]))

DayAheadOnOffChargingStatusNodes = [
    range(n_cons + i * N + DayAheadOnOffChargingStatusIndices[0], n_cons + i * N + DayAheadOnOffChargingStatusIndices[0] + (DayAheadOnOffChargingStatusIndices[-1] - DayAheadOnOffChargingStatusIndices[0]) + 1) for i in range(BATCH_SIZE)]

assert np.array_equal(n_cons + np.array(DayAheadOnOffChargingStatusIndices), np.array(DayAheadOnOffChargingStatusNodes[0]))

In [100]:
# make output nodes indices array
output_nodes = []
for i in range(BATCH_SIZE):
    output_nodes.append(
        np.concatenate(
            (
                np.array(DayAheadBuySellNodes[i]),
                np.array(DayAheadOnOffChargingStatusNodes[i]),
            )
        )
    )

# flatten output_nodes
output_nodes = np.array(output_nodes).flatten()

In [103]:
solution_dict_list[0].keys()

dict_keys(['DayAheadBuySellStatus', 'DayAheadOnOffChargingStatus', 'DayAheadChargingPower', 'DayAheadUtilityPowerOutput', 'SOC', 'output'])

In [ ]:
# output keys are DayAheadBuySellStatus and DayAheadOnOffChargingStatus

In [82]:
"""
Create a pytorch geometric dataset
1. Graph - Pass in edge_index, edge_attr
2. Node - Pass in the node features tensor for x
3. Create a dataset by subclassing PyTorch Geometric's Dataset class. At a minimum you need to implement:

    len - Returns the number of graphs in the dataset
    get - Retrieves a graph object by its index

4. You can also add additional functionality like transforms, downloading data, etc.
"""

class MIPDataset(tg.data.InMemoryDataset):
    def __init__(self, root, input_data_dict, transform=None, pre_transform=None):
        super(MIPDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ["data.pt"]

    def download(self):
        pass

    def process(self):
        data_list = []
        for i in range(n_sample):
            
            # ensure x is of shape (N, 1)
            assert input_data_dict[i]["x"].shape[0] == N
            assert input_data_dict[i]["x"].shape[1] == 1

            data = tg.data.Data(
                x=input_data_dict[i]["x"],
                edge_index=input_data_dict[i]["edge_index"],
                edge_attr=input_data_dict[i]["edge_attr"],
            )
            data_list.append(data)

        data, slices = self.collate(data_list)
        
        torch.save((data, slices), self.processed_paths[0])

In [83]:
"""
Implement a GCN model

Modification to the GCN model:
1. Extend the node embeddings for layer l + 1 by concatenating the node embeddings from layer l. Specifically, we now define the embedding for layer l + 1 to be  ̃ Z(l+1) = (Z(l+1),  ̃ Z(l)), i.e., the concatenation of the matrices row-wise, with  ̃ Z(0) = Z0
2. Apply layer norm at the output of each layer
3.  modification made to a Multi-Layer Perceptron (MLP) function called fθ. 
The original function was a linear mapping followed by a fixed nonlinearity in a standard Graph Convolutional Network (GCN) developed by Kipf and Welling in 2016. 
However, in this paper, the researchers have generalized fθ to be an MLP,
"""

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels):
        super(GCN, self).__init__()

        self.conv1 = tg.nn.GCNConv(
            in_channels, hidden_channels, cached=True, normalize=False
        )
        self.conv2 = tg.nn.GCNConv(
            hidden_channels, hidden_channels, cached=True, normalize=False
        )
        self.conv3 = tg.nn.GCNConv(
            hidden_channels, 1, cached=True, normalize=False
        )

        self.layernorm1 = torch.nn.LayerNorm(hidden_channels)
        self.layernorm2 = torch.nn.LayerNorm(hidden_channels)
        
        self.mlp1 = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, hidden_channels),
        )

        self.mlp2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, hidden_channels),
        )

        self.mlp3 = torch.nn.Sequential(
            torch.nn.Linear(out_channels, out_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels, out_channels),
        )


    def forward(self, x, edge_index, edge_attr):
        
        # concatenate the node embeddings from layer l. Specifically, we now define the embedding for layer l + 1 to be  ̃ Z(l+1) = (Z(l+1),  ̃ Z(l)), i.e., the concatenation of the matrices row-wise, with  ̃ Z(0) = Z0 (the first layer )

        
            # use a prev_x to store the previous layer's node embeddings
        x = self.conv1(x, edge_index, edge_attr)
        x = self.layernorm1(x)
        Z_tilde_0 = self.mlp1(x)

        Z_tilde = self.conv2(Z_tilde_0, edge_index, edge_attr)
        Z_tilde = self.layernorm2(Z_tilde)
        Z_tilde = self.mlp2(Z_tilde)
        Z_tilde = torch.cat([Z_tilde, Z_tilde_0], dim=0)

        Z_tilde = self.conv3(Z_tilde, edge_index, edge_attr)

        # reshape to N
        Z_tilde = Z_tilde.reshape(N)

        # output nodes are the DayAheadOnOffChargingStatusNodes and DayAheadBuySellNodes
        out = Z_tilde[DayAheadOnOffChargingStatusNodes]

        out = self.mlp3(Z_tilde)
                
        out_prob = torch.sigmoid(out)

        return out_prob


In [84]:
# create dataloader
data_root_dir = "./Data/input_data/"
dataset = MIPDataset(data_root_dir, input_data_dict)
dataloader = tg.data.DataLoader(dataset, batch_size=1, shuffle=True)

/opt/homebrew/Caskroom/miniforge/base/envs/me555/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [85]:
for data in dataloader:
    print(data.x[1697159])
    break

tensor(-0.0007, dtype=torch.float64)


In [95]:
# get total number of output nodes. total number of output nodes is the number of nodes in DayAheadOnOffChargingStatusNodes and DayAheadBuySellNodes 
out_channels = len(DayAheadOnOffChargingStatusNodes[0]) * len(DayAheadOnOffChargingStatusNodes) + len(DayAheadBuySellNodes[0]) * len(DayAheadBuySellNodes)


288960

In [69]:
# create model
model = GCN(in_channels=1, out_channels=out_channels, hidden_channels=32)

# create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
print(model)

GCN(
  (conv1): GCNConv(1, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (layernorm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (layernorm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (mlp1): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (mlp2): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (mlp3): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [ ]:
# test model
for data in dataloader:

    # ensure x is of shape (N, 1), if not, reshape, use try except
    # ensure edge_index is of shape (2, E), if not, reshape, use try except
    # ensure edge_attr is of shape (E, 1), if not, reshape, use try except

    try:
        assert data.x.shape[1] == 1

        assert data.edge_index.shape[0] == 2

        assert data.edge_attr.shape[1] == 1

    except:
        data.x = data.x.reshape(-1, 1)
        data.edge_index = data.edge_index.reshape(2, -1)
        data.edge_attr = data.edge_attr.reshape(-1, 1)

    # convert to float
    data.x = data.x.float()
    data.edge_index = data.edge_index.long()
    data.edge_attr = data.edge_attr.float()

    print("Shape of x: ", data.x.shape)
    print("Shape of edge_index: ", data.edge_index.shape)
    print("Shape of edge_attr: ", data.edge_attr.shape)
    out = model(data.x, data.edge_index, data.edge_attr)

Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970])
Shape of edge_attr:  torch.Size([2392970, 1])
Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970])
Shape of edge_attr:  torch.Size([2392970, 1])
Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970])
Shape of edge_attr:  torch.Size([2392970, 1])
Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970])
Shape of edge_attr:  torch.Size([2392970, 1])
Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970])
Shape of edge_attr:  torch.Size([2392970, 1])
Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970])
Shape of edge_attr:  torch.Size([2392970, 1])
Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970])
Shape of edge_attr:  torch.Size([2392970, 1])
Shape of x:  torch.Size([1697160, 1])
Shape of edge_index:  torch.Size([2, 2392970]

KeyboardInterrupt: 

In [217]:
out.shape

torch.Size([3394320, 1])